In [6]:

import torch
import torch.nn.functional as F
from rethink.models import create_model
import rethink.utils as utils
from rethink.preprocess import * 


In [7]:
model_checkpoint = "/media/magalhaes/sound/checkpoints/yolo_1672487331/last1.pth"

In [8]:
human_eval = [
    "Car Truck",
    "Car Truck",
    "Truck",
    "Truck",
    "Car Truck",
    "Bus Truck",
    "Car Bus"
]

In [9]:
audios_arr = [
    "/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 10_25_24.667601+00_00.wav",
    "/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 10_48_52.668143+00_00.wav",
    "/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 11_10_11.777189+00_00.wav",
    "/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 11_11_26.000122+00_00.wav",
    "/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 11_19_54.844896+00_00.wav",
    "/media/magalhaes/schreder_sound/21_12_2022/2022-12-21 11_20_20.511141+00_00.wav",
    "/media/magalhaes/schreder_sound/18_12_2022/2022-12-18 16_49_42.572795+00_00.wav",
]


In [10]:
model = create_model("yolo", 8, device="cuda")
utils.load_checkpoint(model_checkpoint, model)

/home/dferrao/miniconda3/envs/magalhaes/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


TypeError: exceptions must derive from BaseException

In [ ]:
def load_audios(audio_paths):
    loader = Loader(sample_rate=22050)
    trimmer = Trimmer(sample_rate=22050, duration=10)
    padder = Padder(mode="constant", sample_rate=22050, duration=10)
    extractor = LogMelSpectrogramExtractorModel(sample_rate=22050, n_mels=128, length=250, duration=10)
    batch = []
    for audio_path in audio_paths:
        spectrogram = extractor(padder(trimmer(loader(audio_path)))).unsqueeze(0)
        batch.append(spectrogram)
    return torch.cat(batch)

In [ ]:
id_to_class_name = utils.load_id_to_class_name("../rethink/data/schreder.names")

In [ ]:
batch = load_audios(audios_arr).to("cuda")
model.eval()
with torch.no_grad():
    predictions = model(batch)
    predictions = F.softmax(predictions)
    for eval, path, pred in zip(human_eval, audios_arr, predictions):
        print(f"{path}: ")
        print(f"My eval: {eval}")
        for i, v in enumerate(pred):
            print(f"{id_to_class_name[i]} - {v}")

/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 10_25_24.667601+00_00.wav: 
My eval: Car Truck
person - 0.9975630044937134
bicycle - 1.559285919938702e-05
car - 2.4322567696799524e-05
motorcycle - 4.135784365644213e-06
siren - 0.0023461561650037766
bus - 1.364014588034479e-05
truck - 6.691972885164432e-06
horn - 2.6476152925170027e-05
/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 10_48_52.668143+00_00.wav: 
My eval: Car Truck
person - 0.9714928865432739
bicycle - 0.0004001471388619393
car - 0.0002756762260105461
motorcycle - 2.1540818124776706e-05
siren - 0.02735089883208275
bus - 0.00010373005352448672
truck - 0.00018754250777419657
horn - 0.00016759303980506957
/media/magalhaes/schreder_sound/22_9_2022/2022-09-22 11_10_11.777189+00_00.wav: 
My eval: Truck
person - 0.9931941032409668
bicycle - 0.00019670043548103422
car - 9.368908649776131e-05
motorcycle - 1.5795747458469123e-05
siren - 0.0063553196378052235
bus - 3.870458749588579e-05
truck - 3.296986687928438e-05
horn - 

/tmp/ipykernel_497870/3974109014.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = F.softmax(predictions)
